In [1]:
import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import cv2

In [ ]:
# Check if GPU is enabled
print(tf.__version__)
print(tf.test.gpu_device_name())

In [ ]:
%cd c:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101

In [ ]:
!ls food-101/

In [ ]:
os.listdir('images')

In [ ]:
os.listdir('meta')

In [ ]:
!head meta/train.txt

In [ ]:
!head meta/classes.txt

In [ ]:
rows = 17
cols = 6
fig, ax = plt.subplots(rows, cols, figsize=(25,25))
fig.suptitle("Showing one random image from each class", y=1.05, fontsize=24) # Adding  y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
data_dir = "images/"
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    if food_selected == '.DS_Store':
        continue
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected)) # returns the list of all files present in each food category
    food_selected_random = np.random.choice(food_selected_images) # picks one food item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 10)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()
# https://matplotlib.org/users/tight_layout_guide.html

In [10]:
# Helper method to split dataset into train and test folders
def prepare_data(filepath, src, dest):
    classes_images = defaultdict(list)

    # Reading the file paths from the provided text file
    with open(filepath, 'r') as txt:
        paths = [read.strip() for read in txt.readlines()]
        for p in paths:
            food = p.split('/')
            if len(food) == 2:  # Ensure there's a valid path structure
                classes_images[food[0]].append(food[1] + '.jpg')

    # Copying the images from source to destination
    for food in classes_images.keys():
        print(f"\nCopying images into {food}")
        
        food_dest_path = os.path.join(dest, food)
        
        if not os.path.exists(food_dest_path):
            os.makedirs(food_dest_path)
            
        for img_name in classes_images[food]:
            src_image_path = os.path.join(src, food, img_name)
            dest_image_path = os.path.join(food_dest_path, img_name)
            
            if os.path.exists(src_image_path):  # Check if source image exists
                copy(src_image_path, dest_image_path)
            else:
                print(f"Warning: {src_image_path} does not exist.")
                
    print("Copying Done!")

In [ ]:
# Prepare train dataset by copying images from food-101/images to food-101/train using the file train.txt
import os

# Change the working directory to the root if necessary
%cd /

print("Creating train data...")

# Adjusted the paths for the source and destination
train_txt_path = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\meta\train.txt'  # Raw string for file path
images_src_path = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\images'  # Raw string for source path
train_dest_path = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train'  # Raw string for destination path

# Calling the helper function to copy the data
prepare_data(train_txt_path, images_src_path, train_dest_path)


In [ ]:
# Prepare test data by copying images from food-101/images to food-101/test using the file test.txt
import os

print("Creating test data...")

# Adjust the paths for the source and destination
test_txt_path = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\meta\test.txt'  # Path to test.txt
images_src_path = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\images'  # Source path where images are stored
test_dest_path = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test'  # Destination path where test images will be copied

# Calling the helper function to copy the test data
prepare_data(test_txt_path, images_src_path, test_dest_path)


In [ ]:
import os

# Path to the train folder
train_dir = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train'

# Count the total number of files in the train directory and its subdirectories
total_files = sum([len(files) for r, d, files in os.walk(train_dir)])

print(f"Total number of samples in the train folder: {total_files}")


In [ ]:
import os

# Path to the test folder
test_dir = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test'

# Count the total number of files in the test directory and its subdirectories
total_files = sum([len(files) for r, d, files in os.walk(test_dir)])

print(f"Total number of samples in the test folder: {total_files}")


In [15]:
import os

# Change the working directory to C: root in Windows
os.chdir('C:\\')


In [16]:
# List of all 101 types of foods(sorted alphabetically)
del foods_sorted[0] # remove .DS_Store from the list

In [ ]:
foods_sorted

In [18]:
import os
from shutil import copytree, rmtree

# Helper method to create train_mini and test_mini data samples
def dataset_mini(food_list, src, dest):
    # Remove existing mini dataset if it already exists
    if os.path.exists(dest):
        rmtree(dest)  # Remove the destination directory
        print(f"Removed existing directory: {dest}")
    
    # Create the destination directory
    os.makedirs(dest)
    print(f"Created new directory: {dest}")
    
    # Copy selected food categories to the mini dataset
    for food_item in food_list:
        src_path = os.path.join(src, food_item)
        dest_path = os.path.join(dest, food_item)
        
        # Ensure the source food category exists before copying
        if os.path.exists(src_path):
            print(f"Copying images from {src_path} to {dest_path}")
            copytree(src_path, dest_path)
        else:
            print(f"Warning: {src_path} does not exist.")
            
    print("Copying Done!")



In [ ]:
# Define the source and destination paths for train and test mini datasets
food_list = ['apple_pie', 'pizza', 'omelette']

# Source paths for train and test datasets
src_train = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train'
src_test = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test'

# Destination paths for the mini train and test datasets
dest_train = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train_mini'
dest_test = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test_mini'

# Create the mini train dataset
print("Creating train mini dataset...")
dataset_mini(food_list, src_train, dest_train)

# Create the mini test dataset
print("Creating test mini dataset...")
dataset_mini(food_list, src_test, dest_test)


In [ ]:
# Define the food items to include in the mini dataset
food_list = ['apple_pie', 'pizza', 'omelette']

# Source path for the original train dataset
src_train = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train'

# Destination path for the mini train dataset
dest_train = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train_mini'

# Create the mini train dataset
print("Creating train data folder with new classes...")
dataset_mini(food_list, src_train, dest_train)


In [ ]:
import os

# Path to the train_mini folder
train_mini_dir = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train_mini'

# Count the total number of files in the train_mini directory and its subdirectories
total_samples = sum([len(files) for r, d, files in os.walk(train_mini_dir)])

print(f"Total number of samples in train folder: {total_samples}")


In [ ]:
# Define the food items to include in the mini dataset
food_list = ['apple_pie', 'pizza', 'omelette']

# Source path for the original test dataset
src_test = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test'

# Destination path for the mini test dataset
dest_test = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test_mini'

# Create the mini test dataset
print("Creating test data folder with new classes...")
dataset_mini(food_list, src_test, dest_test)


In [ ]:
import os

# Path to the test_mini folder
test_mini_dir = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test_mini'

# Count the total number of files in the test_mini directory and its subdirectories
total_samples_test = sum([len(files) for r, d, files in os.walk(test_mini_dir)])

print(f"Total number of samples in test folder: {total_samples_test}")


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras import backend as K

# Clear previous session
K.clear_session()

# Parameters
n_classes = 3
img_width, img_height = 224, 224
train_data_dir = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\train_mini'
validation_data_dir = r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\test_mini'
nb_train_samples = 2250
nb_validation_samples = 750
batch_size = 16

# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Load the ResNet50 model
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
x = resnet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)

# Final prediction layer
predictions = Dense(n_classes, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

# Compile the model
model = Model(inputs=resnet50.input, outputs=predictions)
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
# Callbacks
checkpointer = ModelCheckpoint(filepath=r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\best_model_3class.keras', verbose=1, save_best_only=True)
csv_logger = CSVLogger(r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\history_3class.log')

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    epochs=30,
    verbose=1,
    callbacks=[csv_logger, checkpointer]
)

# Save the trained model
model.save(r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\model_trained_3class.keras')


In [ ]:
# Display the class-to-index mapping
class_map_3 = train_generator.class_indices
print(class_map_3)


In [25]:
import matplotlib.pyplot as plt

def plot_accuracy(history, title):
    plt.title(title)
    plt.plot(history.history['accuracy'], label='train_accuracy')
    plt.plot(history.history['val_accuracy'], label='validation_accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.show()

def plot_loss(history, title):
    plt.title(title)
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='validation_loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.show()


In [ ]:
# Plotting accuracy
plot_accuracy(history, 'FOOD101-ResNet50')

# Plotting loss
plot_loss(history, 'FOOD101-ResNet50')


In [ ]:
import os
print(os.listdir(r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101'))


In [36]:
model_best = load_model(r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\best_model_3class.keras', compile=False)


In [ ]:
import requests
import os

# Ensure this directory exists
download_directory = r"C:\Users\chinn\Downloads"  # Change this path as needed
os.makedirs(download_directory, exist_ok=True)

def download_image(url, filename):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
        full_path = os.path.join(download_directory, filename)
        with open(full_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {full_path}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {filename}: {e}")

# URLs of the images (Updated with valid links)
download_image("https://queen.com.au/wp-content/uploads/2019/10/Pastel-Rainbow-Cupcakes-WEB.jpg", "cupcakes.jpg")
download_image("https://www.sidechef.com/recipe/b75f5913-ca28-41a7-b985-30ab40045d7a.jpg", "springrolls_new.jpg")  # Changed filename
download_image("https://i0.wp.com/www.amysrecipebook.com/wp-content/uploads/2021/01/pepperonipizza-8-web.jpg", "pizza.jpg")  # Valid pizza URL
download_image("https://c1.staticflickr.com/1/84/262952165_7ba3466108_z.jpg", "garlicbread.jpg")


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the trained model
model_best = load_model(r'C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\best_model_3class.keras', compile=False)

# Function to preprocess images and make predictions
def predict_class(model, image_paths, show_images=False):
    class_map = {0: 'Class_1', 1: 'Class_2', 2: 'Class_3'}  # Update with your actual class names
    
    for image_path in image_paths:
        # Load and preprocess the image
        img = load_img(image_path, target_size=(224, 224))  # Resize to the input shape of your model
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array /= 255.0  # Normalize pixel values

        # Make prediction
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions)

        # Output the result
        print(f"Predicted class for {image_path}: {class_map[predicted_class]}")

        # Optionally display the image
        if show_images:
            import matplotlib.pyplot as plt
            
            plt.imshow(img_array[0])  # Display the image
            plt.title(f"Predicted: {class_map[predicted_class]}")
            plt.axis('off')
            plt.show()

# List of images to test
images = [
    r'C:\Users\chinn\Downloads\cupcakes.jpg',
    r'C:\Users\chinn\Downloads\springrolls_new.jpg',
    r'C:\Users\chinn\Downloads\pizza.jpg',
    r'C:\Users\chinn\Downloads\garlicbread.jpg'
]

# Call the function to predict classes
predict_class(model_best, images, show_images=True)


In [52]:
import os

# Change the current working directory
os.chdir(r"C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101")


In [ ]:
import tensorflow as tf

model_plot_path = r"C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\model_plot.png"
tf.keras.utils.plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)


In [ ]:
import matplotlib.pyplot as plt

def plot_model_summary(model):
    # Get model summary
    summary_str = []
    model.summary(print_fn=lambda x: summary_str.append(x))

    # Debugging: print the summary lines
    print("Model Summary Lines:")
    for line in summary_str:
        print(line)

    # Prepare data for the table
    table_data = []
    for line in summary_str[2:]:  # Skip the first two lines (header and separators)
        parts = line.split()
        if len(parts) >= 3:
            layer_info = parts[:3]  # Take the first three parts (Layer, Output Shape, Param #)
            table_data.append(layer_info)
        else:
            print(f"Unexpected line format: {line}")  # Debugging unexpected formats

    # Check if table_data is empty
    if not table_data:
        print("No data found for the table. Exiting.")
        return

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, len(table_data) * 0.5 + 1))  # Adjust height based on rows

    # Create a table to display model summary
    table = ax.table(cellText=table_data,
                     colLabels=["Layer (type)", "Output Shape", "Param #"],
                     loc='center', cellLoc='center')

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.5, 1.5)  # Scale table size
    ax.axis('off')  # Hide axes

    # Save the plot to a file
    plt.savefig(r"C:\Users\chinn\ml\PRODIGY_ML_05\PRODIGY_ML_05\food-101\model_summary.png", bbox_inches='tight')
    plt.show()

# Call the function to plot model summary
plot_model_summary(model)
